In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

## configurations

### general configurations

In [ ]:
repo_dir = '/home/labs/amit/noamsh/repos/CAR_T'
data_dir_path = Path(repo_dir, "data")

In [ ]:
sys.path.append(repo_dir)

### experiment configuration

to reproduce paper, run this notebook 3 time, while edditing the feature selection parmeters:

    1. use_cell_frequencies = True 
    2. use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono = True 
    3. use_cell_frequencies = True, use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono = True 


In [ ]:
# feature selection
use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono = False # will be used only is above is false
use_cell_frequencies = True

## data loading

### abundance + label

In [ ]:
cell_abundance_dataset_path  = Path(data_dir_path, "cell_type_abundance_by_sample_group_normalized.csv")
cell_abundance_dataset = pd.read_csv(cell_abundance_dataset_path)
cell_abundance_dataset = cell_abundance_dataset.set_index('sample_id')

y = cell_abundance_dataset['response_3m']
cell_abandace = cell_abundance_dataset.drop(columns=['response_3m', 'LDH_prior_tx'])
cell_abandace.index = cell_abandace.index.rename("patient")

print(cell_abandace.shape)
print(cell_abandace.columns)

cell_abandace.head()

In [ ]:
cell_group_abundance_dataset_path  = Path(data_dir_path, "cell_group_abundance_by_sample.csv")
cell_group_abundance = pd.read_csv(cell_group_abundance_dataset_path)
cell_group_abundance = cell_group_abundance[['patient_alias', 'cell_type', 'abundance']].groupby(by=['patient_alias', 'cell_type']).sum()['abundance'].unstack()
cell_group_abundance.index = cell_group_abundance.index.rename("patient")
cell_group_abundance = cell_group_abundance / 100
cell_group_abundance = cell_group_abundance.loc[cell_abandace.index]

cell_group_abundance["NKT"] = cell_group_abundance[["CD4 T", "CD8 T", "NK"]].sum(axis=1)
print(cell_group_abundance.shape)
cell_group_abundance.head()

In [ ]:
cell_abandace = pd.concat([cell_abandace, cell_group_abundance.drop(columns="B")], axis=1)

### baseline

In [ ]:
baseline_data_path = Path(data_dir_path, "patients_baseline_predictions.csv")
baseline_data = pd.read_csv(baseline_data_path)
baseline_data = baseline_data.rename(columns={'Unnamed: 0': 'sample_id'})
baseline_data = baseline_data.set_index('sample_id')

baseline_data.index = baseline_data.index.rename("patient")

print(baseline_data.shape)
print(baseline_data.columns)
baseline_data.head()

In [ ]:
pd.testing.assert_series_equal(baseline_data["TRUE"], y, check_names=False)

### patient metadata

In [ ]:
metadata_path = Path(data_dir_path, "2024_03_04 CART Clinical metadata analysed only V2.xlsx")
patient_metadata = pd.read_excel(metadata_path).dropna(subset="sample_name")
patient_metadata["Product {GIL: 0, NOV:1}"] = patient_metadata["Product"].map({"GIL": 0, "NOV":1})
patient_metadata["Sex {M: 0, F:1}"] = patient_metadata["Sex"].map({"M": 0, "F":1})
patient_metadata["sample_name"] = patient_metadata["sample_name"].apply(lambda x: x.split('/')[0])

# patient_metadata.columns

In [ ]:
patient_map_path = Path(data_dir_path, "sample_names_ids.csv")
patient_map = pd.read_csv(patient_map_path)
patient_map.head()

In [ ]:
patient_metadata = patient_metadata.merge(patient_map, on="sample_name", how="inner", validate="1:1")

In [ ]:
# "Sex {M: 0, F:1}", "Pick_Ferritin"

patient_metadata = patient_metadata[["sample_id",'Day_7_Expansion (CAR T/ml blood)',  "Age", "Product {GIL: 0, NOV:1}"]]
patient_metadata = patient_metadata.set_index("sample_id")
patient_metadata.index = patient_metadata.index.rename("patient")

In [ ]:
print(patient_metadata.shape)
print(patient_metadata.columns)
patient_metadata.head()

### gene pathways

In [ ]:
mye_pathways_path = Path(data_dir_path, "Myeloid_zscore_top10_padj_pathways.csv")
mye_pathways = pd.read_csv(mye_pathways_path)

print(mye_pathways.shape)
print(mye_pathways.columns)
mye_pathways.head()

In [ ]:
mye_pathways['cleaned_pathways-cell_type'] = mye_pathways['cleaned_pathways'] + "-" + mye_pathways['cell_type'] 
mye_pathways_zscores = mye_pathways[['patient', 'zscore', 'cleaned_pathways-cell_type']].groupby(['patient', 'cleaned_pathways-cell_type'])["zscore"].sum().unstack(level=-1)
print(mye_pathways_zscores.shape)
print(mye_pathways_zscores.columns)
mye_pathways_zscores.head()
# mye_pathways_zscores

### merge data

In [ ]:
patient_col = "patient"

cell_abandace.index = cell_abandace.index.astype("string").str.replace("-","_")
mye_pathways_zscores.index = mye_pathways_zscores.index.astype("string").str.replace("-","_")
patient_metadata.index = patient_metadata.index.astype("string").str.replace("-","_")

In [ ]:
mye_pathways_zscores.loc["NOV_20"] = mye_pathways_zscores.mean()

In [ ]:
print("abandace - mye: ", set(cell_abandace.index).difference(set(mye_pathways_zscores.index)))
print("mye - abandace: ", set(mye_pathways_zscores.index).difference(set(cell_abandace.index)))
###
print("metadate - mye: ", set(patient_metadata.index).difference(set(mye_pathways_zscores.index)))
print("mye - metadate: ", set(mye_pathways_zscores.index).difference(set(patient_metadata.index)))
####
print("metadata - abandace: ", set(patient_metadata.index).difference(set(cell_abandace.index)))
print("abandace - metadata: ", set(cell_abandace.index).difference(set(patient_metadata.index)))

print("NOV_06 not in metadata, NOV_20 not in mye, GIL_08 not in abandance")

In [ ]:
all_X = cell_abandace.reset_index().merge(mye_pathways_zscores.reset_index(), how="inner", on=patient_col)

In [ ]:
all_X = all_X.merge(patient_metadata.reset_index(), on=patient_col, how="inner")

In [ ]:
all_feats_names = {
    "mye_pathways": list(mye_pathways_zscores.columns),
    "abundance": list(cell_abandace.columns),
    "metadata": list(patient_metadata.columns)
}

In [ ]:
all_X = all_X.set_index(patient_col)
print(all_X.shape, y.shape)
print(all_X.columns)

In [ ]:
all_X["Product {GIL: 0, NOV:1}"].value_counts()

In [ ]:
class_map = {"R":1, "NR":0}
y = y.map(class_map)

In [ ]:
y = y.loc[all_X.index]
y.value_counts()

## feature_selection

In [ ]:
single_pathway = "HALLMARK_TNFA_SIGNALING_VIA_NFKB-CD16 Mono"

featurs = []
if use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono:
    featurs += [single_pathway]  
if use_cell_frequencies:
    featurs += all_feats_names["abundance"]

X = all_X[featurs]

In [ ]:
X = X.dropna(axis=1)

In [ ]:
B_cat_map = {'B_unknown': 0, 'B_tumor': -1, 'B_healthy': 1}
if 'B_category' in featurs:
    X['B_category'] = X['B_category'].replace(B_cat_map)

### engeneering


In [ ]:
def normalize_feat(X, feat_name):
    X[feat_name] = (X[feat_name] - X[feat_name].mean())/X[feat_name].std()

In [ ]:
if use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono:
    normalize_feat(X, single_pathway)

### view

In [ ]:
print(X.shape)
print(X.columns)
X.head()

## cross validation hp search

In [ ]:
from sklearn.model_selection import train_test_split
from clinical_predictions.optuna_optimization import get_best_model_with_optuna

In [ ]:
from sklearn.decomposition import PCA

def pca_fit_transform_train_transform_test(X_train, X_test, feats_to_pca, pc_name, n_pcs=3):
    assert all([feat in X_train.columns for feat in feats_to_pca]), ValueError("not all feats in X_train")
    assert all([feat in X_test.columns for feat in feats_to_pca]), ValueError("not all feats in X_test")

    pca = PCA(n_components=n_pcs)
    pca_X_train = pca.fit_transform(X_train[feats_to_pca])
    pca_X_test = pca.transform(X_test[feats_to_pca])

    pc_names = [f"{pc_name}_pc_{i}" for i in range(n_pcs)]
    X_train_transformed = pd.concat([X_train.drop(columns=feats_to_pca), 
                                       pd.DataFrame(pca_X_train, columns=pc_names, index=X_train.index)],
                                      axis=1)
    X_test_transformed = pd.concat([X_test.drop(columns=feats_to_pca), 
                                        pd.DataFrame(pca_X_test, columns=pc_names, index=X_test.index)],
                                    axis=1)
    
    return X_train_transformed, X_test_transformed


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
best_model, best_trail = get_best_model_with_optuna(X_train, y_train, precision_alpha = 0, n_trials=100)

In [ ]:
print(best_trail.user_attrs["best_booster"])
pd.DataFrame(best_trail.user_attrs["scores"])

## evaluation - on train set

#### prep

In [ ]:
from evaluation.visualizations import plot_ROC_PRauc_CM_stem

In [ ]:
def predict_train_test(model, X_1, X_2):
    y_train_pred = model.predict(X_1)
    y_train_proba = model.predict_proba(X_1)[:,1]
    y_test_pred = model.predict(X_2)
    y_test_proba = model.predict_proba(X_2)[:,1]
    return y_train_pred, y_train_proba, y_test_pred, y_test_proba

from sklearn.base import clone
    
def predict_using_all_other(model, X, y, return_trained_models=False, use_pca_of_feats=None, pc_name="feature"):
    
    y_pred_proba = {}
    y_pred = {}
    trained_models_dict = {}
    for pid in y.index:
        _model = clone(model)
        X_train = X.drop(pid)
        y_train = y.drop(pid)

        X_pred = X.loc[pid]
        X_pred = pd.DataFrame(X_pred).T
        
        if not use_pca_of_feats is None:
            X_train, X_pred = pca_fit_transform_train_transform_test(X_train, X_pred, feats_to_pca=use_pca_of_feats, pc_name=pc_name)
        _model.fit(X_train, y_train)
        
        pred = _model.predict(X_pred)[0]
        proba_ref = _model.predict_proba(X_pred)[0][1]
        y_pred[pid] = pred
        y_pred_proba[pid] = proba_ref
        trained_models_dict[pid] = _model
    
    y_proba_pred = pd.Series(y_pred_proba)
    y_pred = pd.Series(y_pred)
    if return_trained_models:
        return y_pred, y_proba_pred, trained_models_dict
    return y_pred, y_proba_pred
    # y_proba_pred.hist()

### response

In [ ]:
best_model

In [ ]:
best_model.fit(X_train, y_train)
y_train_pred, y_train_proba, y_test_pred, y_test_proba = predict_train_test(best_model, X_train, X_test)
plot_ROC_PRauc_CM_stem(y_train, y_train_pred, y_train_proba, pos_label=1)

In [ ]:
y_train_pred, y_train_proba, models_dict= predict_using_all_other(best_model, X_train, y_train, return_trained_models=True)
plot_ROC_PRauc_CM_stem(y_train, y_train_pred, y_train_proba, pos_label=best_model.classes_[1])

In [ ]:
df = "NONE"
num_highest = 3
try:
    bad_patients = y_train_proba[y_train==0].sort_values().iloc[-num_highest:]
    good_patient = y_train_proba[y_train==1].sort_values().iloc[-num_highest:]
    assert all(y[bad_patients.index] == 0)
    assert all(y[good_patient.index] == 1)
    df = pd.concat([X.loc[bad_patients.index], X.loc[good_patient.index]])
except:
    print("ERROR")
df

In [ ]:
try:
    plot_df = X.copy()
    plot_df["y_pred"] = y_train_proba
    plot_df["y_true"] = y
    # featurs = single_pathway, "HALLMARK_INFLAMMATORY_RESPONSE-CD16 Mono", # "HALLMARK_APICAL_JUNCTION-CD16 Mono",
    fig = px.scatter(plot_df.reset_index(), x=single_pathway,
                     y= "CD16 Mono", 
                     color="y_pred",
                     # size='petal_length',
                     hover_data=['patient', "y_pred", "B", "y_true"]
                    )
    fig.show()
except ValueError:
    pass

## explainability and error analisys

In [ ]:
import shap

def print_shap_plots(model, X):
    try: # tree
        explainer = shap.TreeExplainer(model)
    except:
        try: # kernel
            explainer = shap.Explainer(model, X) 
        except:
            explainer = shap.KernelExplainer(model.predict, X)
    shap_values = explainer(X)
    if len(shap_values.shape) >2:
        shap_values = shap_values[:,:,1]
    shap.plots.beeswarm(shap_values)
    shap.plots.bar(shap_values, max_display=5)
    return shap_values


### response

In [ ]:
_ = print_shap_plots(best_model, X_train)
_ = print_shap_plots(best_model, X_test)

In [ ]:
## test

In [ ]:
import numpy as np
from sklearn import metrics
def print_metrics(y_true, y_score):
    report = metrics.classification_report(y_true, y_score, output_dict=True)
    print_report = {
        "R precision": round(report['1']['precision'], 3),
        "NR precision": round(report['0']['precision'], 3),
        'accuracy': round(report['accuracy'],3)
    }
    print_repors_list = [f"{k}: {v}" for k,v in print_report.items()]
    print("\n".join(print_repors_list))

In [ ]:
print_metrics(y_test, y_test_pred)
plot_ROC_PRauc_CM_stem(y_test, y_test_pred, y_test_proba, pos_label=best_model.classes_[1])

In [ ]:
pca_feats = None
    
y_pred, y_proba = predict_using_all_other(best_model, X, y, use_pca_of_feats=pca_feats, pc_name="mye_pathways")
print_metrics(y, y_pred)
plot_ROC_PRauc_CM_stem(y, y_pred, y_proba, flip_stem=True, use_all_score_range=True)

In [ ]:
df = "NONE"
num_highest = 3
try:
    bad_patients = y_proba[y==0].sort_values().iloc[-num_highest:]
    good_patient = y_proba[y==1].sort_values().iloc[-num_highest:]
    assert all(y[bad_patients.index] == 0)
    assert all(y[good_patient.index] == 1)
    df = pd.concat([X.loc[bad_patients.index], X.loc[good_patient.index]])
except:
    print("ERROR")
df

## hard test - external validation

### load data

#### cell type + y + product

In [ ]:
haradvala_cell_freq = pd.read_csv(Path(data_dir_path, 'haradvala_cell_type_abundance_by_response_complete_renamed.csv'))
haradvala_cell_freq = haradvala_cell_freq.set_index('sample_id')

haradvala_cell_freq = haradvala_cell_freq.rename(columns={"Product":"Product {GIL: 0, NOV:1}"})

class_map = {"R":1, "NR":0}
y_hard_test = haradvala_cell_freq["response"].map(class_map)
y_hard_test.index = y_hard_test.index.rename("patient")
haradvala_cell_freq = haradvala_cell_freq.drop(columns="response")
haradvala_cell_freq.index = haradvala_cell_freq.index.rename("patient")

haradvala_cell_freq.shape, haradvala_cell_freq.columns
# y_hard_test

In [ ]:
5e-1

In [ ]:
y_hard_test = y_hard_test.loc[haradvala_cell_freq.index]
y_hard_test.shape

In [ ]:
haradvala_metadata = haradvala_cell_freq[["Product {GIL: 0, NOV:1}"]]
haradvala_cell_freq = haradvala_cell_freq.drop(columns="Product {GIL: 0, NOV:1}")

In [ ]:
haradvala_cell_freq.head()

#### cell group

In [ ]:
raw_group_df = pd.read_csv(Path(data_dir_path, 'Harvdhala_cell_group_abundance_by_response.csv'))
group_df = raw_group_df[["patient_alias", "cell_type", "abundance"]].groupby(by=["patient_alias", "cell_type"]).sum()[ "abundance"].unstack(level=-1)
group_df.index = group_df.index.rename("patient")
group_df = group_df.rename(columns={'myeloid': 'Myeloid'})
group_df = group_df/100
group_df.head()

#### mye pathways

In [ ]:
haradvala_mye_pathways = pd.read_csv(Path(data_dir_path, 'Myeloid_Modulescore_all_pathways5genes.csv'))
haradvala_mye_pathways['cell_type'] = haradvala_mye_pathways['cell_type'].apply(lambda x: " ".join(x.split(".")[::-1]))
haradvala_mye_pathways['cleaned_pathways-cell_type'] = haradvala_mye_pathways['cleaned_pathways'] + "-" + haradvala_mye_pathways['cell_type'] 
haradvala_mye_pathways = haradvala_mye_pathways[['Patient', 'zscore', 'cleaned_pathways-cell_type']].groupby(['Patient', 'cleaned_pathways-cell_type'])["zscore"].sum().unstack(level=-1)
haradvala_mye_pathways.index = haradvala_mye_pathways.index.rename("patient")
haradvala_mye_pathways.index = pd.Series(haradvala_mye_pathways.index).apply(lambda x: f"Patient{int(x[-2:])}-Baseline")

haradvala_mye_pathways.head()

In [ ]:
if use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono:
    normalize_feat(haradvala_mye_pathways, single_pathway)

#### combine

In [ ]:
all_X_hard_test= pd.concat([haradvala_cell_freq.drop(columns="B"), group_df , haradvala_mye_pathways[single_pathway]], axis=1)

In [ ]:
X_hard_test = all_X_hard_test.copy()
if not use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono:
    X_hard_test = X_hard_test.drop(columns=single_pathway)
common_feats = list(set(X).intersection(set(X_hard_test)))
                    
X_trasnformed = X[common_feats]
X_hard_test_transformed = X_hard_test[common_feats]

X_hard_test_transformed.columns

### evaluate

In [ ]:
best_model.fit(X_trasnformed,y)
y_hard_test_pred = best_model.predict(X_hard_test_transformed)
y_hard_test_proba = best_model.predict_proba(X_hard_test_transformed)[:,1]
plot_ROC_PRauc_CM_stem(y_hard_test, y_hard_test_pred, y_hard_test_proba)

In [ ]:
_ = print_shap_plots(best_model, X_hard_test_transformed)
_ = print_shap_plots(best_model, X_trasnformed)

## save all results

In [ ]:
from evaluation.experiment_managment import generate_experiment_name

experiment_name = generate_experiment_name(use_cell_frequencies=use_cell_frequencies,
                                           use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono=use_TNFA_SIGNALING_VIA_NFKB_CD16_Mono,
                                           )
experiment_name

In [ ]:
all_exp_data = {
    "X": X,
    "y": y,
    'patient_map': patient_map.drop(columns='Unnamed: 0').rename(columns={'sample_id': patient_col}),
    'features': featurs,
    'model': best_model,
    "loocv": {
        'y': y,
        'y_proba': y_proba,
        'y_pred':y_pred
    },
    'test': {
        'X_train':X_train, 
        'X_test': X_test,
        'y':y_test,
        'y_proba':y_test_proba,
        'y_pred':y_test_pred
    },
    'external_eval': {
        "X_train": X_trasnformed,
        "X_test": X_hard_test_transformed,
        'y': y_hard_test, 
        'y_proba': y_hard_test_proba,
        
        'y_pred':y_hard_test_pred
    }
}

In [ ]:
import pickle

In [ ]:
results_path  = Path(data_dir_path, f"{experiment_name}.pkl")
with open(results_path, 'wb') as handle:
    pickle.dump(all_exp_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(results_path, 'rb') as handle:
    loaded_results = pickle.load(handle)
loaded_results.keys()